# 1 часть -- Импорт библиотек

In [27]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

Анализ данного датасета о студентах, обучающихся на курсах математики и португальского языка, может быть полезен по нескольким причинам:

Предсказание успеваемости: Датасет предоставляет множество характеристик студентов, которые могут быть использованы для предсказания их итоговых оценок (G3). Это может помочь образовательным учреждениям выявлять студентов, которым требуется дополнительная поддержка, и разрабатывать программы помощи.

Исследование факторов, влияющих на успеваемость: Включенные в датасет переменные, такие как уровень образования родителей, время, проведенное на учебу, доступ к интернету и наличие дополнительных занятий, позволяют исследовать, какие из этих факторов имеют наибольшее влияние на академическую успеваемость. Это может помочь в разработке стратегий для улучшения образовательных результатов.

Социальные и демографические аспекты: Датасет содержит информацию о социально-экономическом фоне студентов, что позволяет исследовать, как такие факторы, как пол, место жительства (город или село), размер семьи и тип жилья, влияют на успеваемость. Это может помочь в выявлении неравенства в образовании и разработке более инклюзивных образовательных стратегий.

Анализ потребностей студентов: Данные о дополнительных образовательных поддержках, свободном времени, хобби и отношениях с семьей могут помочь в понимании общего состояния благополучия студентов. Это может быть важно для создания более эффективных программ поддержки и благосостояния.

Влияние стресса и алкоголя: Данные о потреблении алкоголя в будние и выходные дни могут быть использованы для анализа влияния на успеваемость и здоровье студентов. Это может помочь в разработке программ по улучшению здоровья и профилактике зависимостей среди молодежи.

Сравнение успеваемости между различными группами: Сравнение успеваемости студентов из разных школ или с различными социально-экономическими фонами может выявить важные различия и тенденции, что может способствовать разработке более адаптированных образовательных программ.


Португалия находится на 16 месте по количеству людей, пьющих ежедневно

In [28]:
df = pd.read_csv('student-mat.csv')

display(df)

df.to_excel('B.xlsx')

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,5,5,4,4,5,4,11,9,9,9
391,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,3,14,16,16
392,MS,M,21,R,GT3,T,1,1,other,other,...,5,5,3,3,3,3,3,10,8,7
393,MS,M,18,R,LE3,T,3,2,services,other,...,4,4,1,3,4,5,0,11,12,10


In [29]:
r_d = {'F': 'Женщина', 'M': 'Мужчина'}
df['sex'] = df['sex'].replace(r_d)
a2 = df.value_counts('sex').reset_index()
g2 = px.pie(a2, names='sex', values='count', labels={'sex': 'Пол', 'count':  'Кол-во'})
g2.update_layout(title = {'text': 'Распределение студентов по полу', 'x': 0.5})
g2.update_legends()
g2.show()




In [30]:
a3 = df.groupby(['age', 'sex']).size().reset_index().rename(columns = {0: 'Кол-во'})
print(a3)
g3 = px.sunburst(a3, path=['sex','age'], values = 'Кол-во', labels={'labels': 'Возраст', 'parent': 'Пол'}, width=800, height=1000)
g3.show()

    age      sex  Кол-во
0    15  Женщина      38
1    15  Мужчина      44
2    16  Женщина      54
3    16  Мужчина      50
4    17  Женщина      58
5    17  Мужчина      40
6    18  Женщина      43
7    18  Мужчина      39
8    19  Женщина      14
9    19  Мужчина      10
10   20  Женщина       1
11   20  Мужчина       2
12   21  Мужчина       1
13   22  Мужчина       1


Очень много студентов в возрасте от 16 до 17 лет. В основном преобладают женщины


In [31]:
# Употребление алкоголя всего (выходные + рабочие дни)
df['Cons'] = df['Dalc'] + df['Walc']
scaler = MinMaxScaler(feature_range=(1, 5))
df['Cons'] = scaler.fit_transform(df[['Cons']])
display(df['Cons'])

0      1.0
1      1.0
2      2.5
3      1.0
4      1.5
      ... 
390    4.5
391    3.5
392    3.0
393    3.5
394    3.0
Name: Cons, Length: 395, dtype: float64

In [32]:
a1 = df['Cons'].value_counts().reset_index()
a1 = a1.sort_values('count', ascending=False)
display(a1)

g1 = px.bar(a1, x='Cons', y = 'count', labels={'Cons': 'Степень', 'count': 'Кол-во'})
g1.update_layout(xaxis_title = 'Степень употребления алкоголя', yaxis_title = 'Количество студентов', title = {'text': 'Количество студентов, употребляющих алкоголь (1 - мало, 5 - очень много)', 'x': 0.5})
g1.show()


,Cons,count
0,1.0,150
1,1.5,66
2,2.0,60
3,2.5,45
4,3.0,35
5,3.5,17
6,5.0,9
7,4.0,9
8,4.5,4


Студенты в основном немного употребляют алкоголя, где-то 1-2 раза в неделю

In [33]:
a11 = df[['address', 'Cons', 'sex']]

g11 = px.histogram(a11, x = 'address', y = 'Cons', color='sex')

g11.update_layout(title = {'text': 'Распределение употребления алкоголя по адресу жительства и полу', 'x': 0.5})

g11.show()


В основном пьют студенты, которые живут в городе. Там больше клубов/баров, и вомзожности выпить всяко больше, чем в деревенских районах

In [34]:
fig = px.box(df, x='school', y='Cons', title='Употребление алкоголя в зависимости от типа школы',
             labels={'school': 'Тип школы', 'Dalc': 'Уровень употребления алкоголя (будни)'})

# Показать график
fig.show()


In [35]:
fig = px.box(df, x='school', y='G3', title='Распределение итоговых оценок (G3) в зависимости от школы',
             labels={'school': 'Тип школы', 'G3': 'Итоговая оценка (G3)'})

fig.show()

В школе GP употребляют алкоголь реже (если судить по медиане) и, как итог, оценки лучше, чем в другой школе

В школе GP в среднем лучше пишут экзамен

In [36]:
fig = px.pie(df, names='Fedu', values='Cons')

fig.update_layout(title = {'text': 'Распределение употребления алкоголя по уровню образования отца (0 - нету, 4 - высшее)', 'x': 0.5})
fig.show()

In [37]:
fig = px.pie(df, names='Medu', values='Cons')

fig.update_layout(title = {'text': 'Распределение употребления алкоголя по уровню образования матери (0 - нету, 4 - высшее)', 'x': 0.5})
fig.show()

Удивительная тенденция - если у 2 родителей есть высшее образование, то ребенок будет пить...

In [38]:
g12 = px.histogram(df, x = 'romantic', facet_col='internet', y = 'Cons', color = 'sex', barmode='stack')

g12.update_layout(title = {'text': 'Распределение употребления алкоголя в зависимости от состояния романтики и интернета', 'x': 0.5}, yaxis_title = 'Кол-во учеников')

g12.show()

Если у многих студентов есть интернет и нету романтических отношений  -- то студенты будут чаще пить

In [39]:
g12 = px.histogram(df, x = 'goout', facet_col='internet', y = 'Cons', color = 'sex', labels={'goout': 'Частота встреч с друзьями'})

g12.update_layout(title = {'text': 'Распределение  выпитого алкоголя от встреч с друзьями и наличия интернета', 'x': 0.5}, yaxis_title = 'Кол-во учеников')

g12.show()

Учащиеся, у которых есть интернет -- чаще собираются с друзьями и чаще пьют алкоголь 

In [40]:
g12 = px.histogram(df, x = 'goout', y = 'freetime', color = 'sex', labels={'goout': 'Частота встреч с друзьями'})

g12.update_layout(title = {'text': 'Распределение свободного времени от встреч с друзьями', 'x': 0.5}, yaxis_title = 'Кол-во учеников')

g12.show()

Если у студентов достаточно свободного времени, то, вероятнее всего, они будут гулять с друзьями

In [41]:
g1 = px.histogram(df, x='G3', labels={'G3': 'Оценка за экзамен', 'count': 'Кол-во учеников'}, color = 'sex')
g1.update_layout(title = {'text': 'Распределение оценок за финальный экзамен', 'x': 0.5}, yaxis_title = 'Кол-во учеников')
g1.show()

Только 1 человек (мужчина) написал на максимальный балл ☠️
Как видно, болеее высокие оценки могут получить мужчины-суденты, нежели чем студентки

In [42]:
g5 = px.histogram(df, x='goout', facet_col='Cons', color='sex')
g5.update_layout(title ={'text': 'Зависимость употребления алкоголя от степени обзения с друзьями', 'x': 0.5})
g5.update_traces(marker=dict(line=dict(width=1, color='DarkSlateGrey')))
g5.show()


Если иногда гулять с друзьями, то верояность того, что студент будет пить увеличивается

In [43]:
a4 = df[['Cons', 'romantic']]
a5 = a4.groupby(['romantic', 'Cons']).size().reset_index().rename(columns={0: 'Кол-во'})
print(a5)


g6 = px.sunburst(a5, path=['romantic','Cons'], values = 'Кол-во', labels={'parent': 'романтика', 'labels': 'употребление'}, width=900, height=1000)
g6.update_layout(title = {'text':  'Количество студентов, пьющих алкоголь и состоящие в романтических отношениях', 'x': 0.5})
g6.show()

   romantic  Cons  Кол-во
0        no   1.0     100
1        no   1.5      44
2        no   2.0      35
3        no   2.5      31
4        no   3.0      28
5        no   3.5      13
6        no   4.0       6
7        no   4.5       2
8        no   5.0       4
9       yes   1.0      50
10      yes   1.5      22
11      yes   2.0      25
12      yes   2.5      14
13      yes   3.0       7
14      yes   3.5       4
15      yes   4.0       3
16      yes   4.5       2
17      yes   5.0       5


Студенты, которые не состоят в романтических отношениях вероятнее всего будут пить 🥴

In [44]:
g9 = px.histogram(df, x='Cons', facet_col='freetime', color='sex',
                  labels={'freetime': 'Свободное время', 'Cons': 'Употребление'},
                  nbins=20)

g9.update_layout(title = {'text': 'Распределение употребления алкоголя в зависимости от свободного времени', 'x': 0.5}, yaxis_title = 'Кол-во учеников')
g9.show()


Если есть немного времени -- то студенты могут и выпить)

In [45]:
a7 = df[['health', 'Cons', 'sex']]

g7 = px.histogram(a7, x = 'Cons', y = 'health', facet_col='sex', color='sex')

g7.update_layout(title = {'text': 'Зависимость употребления алкоголя от здоровья', 'x': 0.5}, yaxis_title = 'Кол-во учеников')
g7.show()

Удивитильно -- но ничто не останавливает студентов. Даже когда они плохо себя чувствуют -- все равно могут выпить

In [46]:
a8 = df[['age', 'sex', 'Cons', 'famsize']]

g8 = px.histogram(a8, x='age', y='Cons', facet_col='sex', color='famsize')

g8.update_layout(title = {'text': 'Зависимость употребления алкоголя от возраста и размера семьи', 'x': 0.5}, yaxis_title = 'Кол-во учеников')
g8.show()

Если в семье > 3 членов, то студенты более склоны пить, особенно в 16-17 лет

Также в основном алкоголь употребляют в 16-17 лет, потом тенденция спадает и подростки пбют меньше

In [47]:
g10 = px.box(df, x='studytime', y='G3', color='sex', labels={'studytime': 'Время на учебу', 'G3': 'Оценка за итоговый экзамен'})
g10.update_layout(title={'text': 'Зависимость итоговых оценок от времени, уделенного учебе', 'x': 0.5}, yaxis_title='Оценка за экзамен')
g10.show()


Общая картина:

Независимо от того, сколько времени студенты уделяют учебе (1 — минимальное время, 4 — максимальное), распределение оценок сохраняет похожие характеристики.
Оценки имеют широкий разброс, но медиана в основном находится в диапазоне 10–15 баллов.
Половые различия:

Мужчины: для всех категорий времени на учёбу медиана оценок выше, чем у женщин. Особенно заметно это для тех, кто выделяет на учебу меньше времени (категория 1).
Женщины: хотя у них медиана ниже, они демонстрируют более стабильное распределение оценок в сравнении с мужчинами, особенно для тех, кто выделяет больше времени (категория 4).
Время на учёбу и результат:

Интересно, что увеличение времени, уделенного учёбе, не показывает явной и однозначной тенденции к повышению оценок. Например, у тех, кто уделяет учебе минимальное время (категория 1), медианные значения оценок близки к тем, кто выделяет больше времени (категории 3 и 4).

Важно отметить, что те, кто выделяет больше времени на учебу (категории 3 и 4), имеют больший разброс оценок. Это может говорить о том, что увеличение времени на учебу не всегда приводит к лучшим результатам.

Что можно сказать:

Время, уделённое учебе, не оказывает существенного влияния на улучшение итоговых оценок, по крайней мере, с очевидной корреляцией.

Мужчины чаще получают более высокие оценки в среднем, особенно при меньшем количестве затраченного времени.

Женщины демонстрируют более стабильное распределение результатов, особенно при увеличении времени на учёбу.

In [48]:
g11 = px.histogram(df, x='absences', y='G3', color='sex', labels={'absences': 'Количество пропусков', 'G3': 'Оценка за итоговый экзамен'})
g11.update_layout(title={'text': 'Зависимость оценок от количества пропущенных занятий', 'x': 0.5}, yaxis_title='Оценка за экзамен')
g11.show()


большее количество пропусков, как правило, связано с низкими оценками. Однако значительная часть студентов (особенно с небольшим количеством пропусков) имеют различные результаты. <br>
Половые различия: мужчины склонны пропускать больше занятий, но зависимости между полом и оценками при одинаковом уровне пропусков не наблюдается.

In [83]:
print(df)
a = px.histogram(df, x='sex', y='age', nbins=4, facet_col='school', color = 'sex')
a.update_legends(title = {'text': 'пол'})
a.show()

     school  sex  age  address  famsize  Pstatus  Medu  Fedu  Mjob  Fjob  ...  \
0         0    0   18        1        0        0     4     4     0     4  ...   
1         0    0   17        1        0        1     1     1     0     2  ...   
2         0    0   15        1        1        1     1     1     0     2  ...   
3         0    0   15        1        0        1     4     2     1     3  ...   
4         0    0   16        1        0        1     3     3     2     2  ...   
..      ...  ...  ...      ...      ...      ...   ...   ...   ...   ...  ...   
390       1    1   20        1        1        0     2     2     3     3  ...   
391       1    1   17        1        1        1     3     1     3     3  ...   
392       1    1   21        0        0        1     1     1     2     2  ...   
393       1    1   18        0        1        1     3     2     3     2  ...   
394       1    1   19        1        1        1     1     1     2     0  ...   

     freetime  goout  Dalc 

In [50]:
a = df
m = LabelEncoder()
for i in a.columns:
    if a[i].dtype == 'object':
        a[i] = m.fit_transform(a[i])

print(a.info())
fig4 = px.imshow(a.corr(), title="Матрица корреляции", width = 1000, height=1000)
fig4.show()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 34 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   school      395 non-null    int32  
 1   sex         395 non-null    int32  
 2   age         395 non-null    int64  
 3   address     395 non-null    int32  
 4   famsize     395 non-null    int32  
 5   Pstatus     395 non-null    int32  
 6   Medu        395 non-null    int64  
 7   Fedu        395 non-null    int64  
 8   Mjob        395 non-null    int32  
 9   Fjob        395 non-null    int32  
 10  reason      395 non-null    int32  
 11  guardian    395 non-null    int32  
 12  traveltime  395 non-null    int64  
 13  studytime   395 non-null    int64  
 14  failures    395 non-null    int64  
 15  schoolsup   395 non-null    int32  
 16  famsup      395 non-null    int32  
 17  paid        395 non-null    int32  
 18  activities  395 non-null    int32  
 19  nursery     395 non-null    i

In [51]:
print(df)

     school  sex  age  address  famsize  Pstatus  Medu  Fedu  Mjob  Fjob  ...  \
0         0    0   18        1        0        0     4     4     0     4  ...   
1         0    0   17        1        0        1     1     1     0     2  ...   
2         0    0   15        1        1        1     1     1     0     2  ...   
3         0    0   15        1        0        1     4     2     1     3  ...   
4         0    0   16        1        0        1     3     3     2     2  ...   
..      ...  ...  ...      ...      ...      ...   ...   ...   ...   ...  ...   
390       1    1   20        1        1        0     2     2     3     3  ...   
391       1    1   17        1        1        1     3     1     3     3  ...   
392       1    1   21        0        0        1     1     1     2     2  ...   
393       1    1   18        0        1        1     3     2     3     2  ...   
394       1    1   19        1        1        1     1     1     2     0  ...   

     freetime  goout  Dalc 